In [ ]:
import sys
import numpy as np
import sklearn as sklearn
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import pairwise_distances
from sklearn import cross_validation as cv
import statistics

def loadTrainFile():
    userID = []
    itemID = []
    rating = []
    with open("train.csv") as f:
        for i,line in enumerate(f):
            if(i>0):
                row = line.strip().split(',')
                
                if(row[2]!='99.0'):
                    userID.append(round(float(row[0])))
                    itemID.append(round(float(row[1])))
                    rating.append(float(row[2]))
                
    return userID, itemID, rating

def loadTestFile():
    userID = []
    itemID = []
    with open("test.csv") as f:
        for i,line in enumerate(f):
            if(i>0):
                row = line.strip().split(',')
                userID.append(round(float(row[0])))
                itemID.append(round(float(row[1])))
                
    return userID, itemID


#############Data#############
train_userID, train_itemID, train_rating = loadTrainFile()
test_userID, test_itemID = loadTestFile()

train_data, test_data = cv.train_test_split(df, test_size=0.25)

#############User-User#############
user_item_ratings = np.zeros([10000,100])
user_item_has_ratings = np.full([10000,100], False)
for i, r in enumerate(train_rating):
    user_item_ratings[train_userID[i]-1][train_itemID[i]-1]=r
    user_item_has_ratings[train_userID[i]-1][train_itemID[i]-1]=True

for i,ratings in enumerate(user_item_ratings):
    n = user_item_has_ratings[i].tolist().count(True)
    mean = ratings.sum()/n
    for j in range(len(ratings)):
        if(user_item_has_ratings[i][j]):
            user_item_ratings[i][j]-=mean
print(user_item_ratings)
print(user_item_ratings.sum(axis=1))
print(user_item_has_ratings)

In [ ]:
user_similarity = np.full([10000,10000],1)
user_similarity = user_similarity-pairwise_distances(user_item_ratings, metric='cosine')
print(user_similarity)
user_similar_userID = []
sort_index = np.argsort(-user_similarity,axis=1)
print(sort_index)


In [ ]:
test_output=[['user_id-item_id','rating']]
for i in range(len(test_userID)):
    user_idx = test_userID[i]-1
    item_idx = test_itemID[i]-1
    n = 0
    similarity_sum = 0.0
    weighted_rate = 0.0
    for neighbor in sort_index[user_idx]:
        if(neighbor == user_idx and user_item_has_ratings[user_idx][item_idx]):
            value = user_item_ratings[user_idx][item_idx]
            break
        if(user_item_has_ratings[neighbor][item_idx]):
            weighted_rate += user_item_ratings[neighbor][item_idx]*user_similarity[user_idx][neighbor]
            similarity_sum += abs(user_similarity[user_idx][neighbor])
            n+=1
            if(n==10):
                weighted_rate /= similarity_sum
                break
            
    entry = str(test_userID[i])+'-'+str(test_itemID[i])
    value = weighted_rate
    test_output.append([entry,value])
np.savetxt("output.csv", np.array(test_output, dtype=np.str), fmt='%s,%s', delimiter=",")

In [1]:
#############Item-Item#############
train_userID, train_itemID, train_rating = loadTrainFile()
test_userID, test_itemID = loadTestFile()
 
item_user_ratings = np.zeros([100,10000])
item_user_has_ratings = np.full([100,10000], False)
for i, r in enumerate(train_rating):
    item_user_ratings[train_itemID[i]-1][train_userID[i]-1]=r
    item_user_has_ratings[train_itemID[i]-1][train_userID[i]-1]=True

#minus mean
item_mean = np.zeros([100])
for i,ratings in enumerate(item_user_ratings):
    n = item_user_has_ratings[i].tolist().count(True)
    mean = ratings.sum()/n
    item_mean[i] = mean
    for j in range(len(ratings)):
        if(item_user_has_ratings[i][j]):
            item_user_ratings[i][j]-=mean
print("item_user_ratings")
print(item_user_ratings)
print(item_user_ratings.sum(axis=1))
print(item_user_has_ratings)

item_similarity = np.full([100,100],1)
item_similarity = item_similarity-pairwise_distances(item_user_ratings, metric='cosine')
print("item_similarity")
print(item_similarity)
sort_index = np.argsort(-item_similarity,axis=1)
print(sort_index)
print(item_mean)

NameError: name 'loadTrainFile' is not defined

In [ ]:
test_output=[['user_id-item_id','rating']]
for i in range(len(test_userID)):
    user_idx = test_userID[i]-1
    item_idx = test_itemID[i]-1
    n = 0
    similarity_sum = 0.0
    weighted_rate = 0.0
#     print('user_idx ', user_idx, 'item_idx ', item_idx)
    if(item_user_has_ratings[item_idx][user_idx]):
        value = item_user_ratings[item_idx][user_idx]
    else:
        for neighbor in sort_index[item_idx]:
            if(item_user_has_ratings[neighbor][user_idx]):
#                 print('neighbor ', neighbor, 'sim=', item_similarity[item_idx][neighbor])
                weighted_rate += item_user_ratings[neighbor][user_idx]*item_similarity[item_idx][neighbor]
                similarity_sum += abs(item_similarity[item_idx][neighbor])
                n+=1
            if(n==15 or item_similarity[item_idx][neighbor]< 0):
                weighted_rate /= similarity_sum
                value = weighted_rate
                break

    entry = str(test_userID[i])+'-'+str(test_itemID[i])
    test_output.append([entry,value])
np.savetxt("output.csv", np.array(test_output, dtype=np.str), fmt='%s,%s', delimiter=",")

In [ ]:

joke_ratings = np.full([100, 10000], 99)
for i, r in enumerate(rating):
    joke_ratings[itemID[i]-1][userID[i]-1]=r

joke_mean_ratings = np.zeros([100])
joke_people_num = np.zeros([100], dtype=int)
for i, per_joke_ratings in enumerate(joke_ratings):
    m = 0
    n = 0
    for score in per_joke_ratings:
        if(score!=99):
            m+=score
            n+=1
    m/=n
    joke_mean_ratings[i]=m
    joke_people_num[i]=n

print(joke_mean_ratings)

users_vec = np.zeros([10000,100])
for i,vec in enumerate(users_vec):
    users_vec[i] = joke_mean_ratings

for i, r in enumerate(rating):
    users_vec[userID[i]-1][itemID[i]-1]=r
print(users_vec)


# plt.hist(joke_ratings[6], bins =  [-10,-8,-6,-4,-2,0,2,4,6,8,10]) 
# plt.title("joke7") 
# plt.show()

# plt.hist(joke_ratings[99], bins =  [-10,-8,-6,-4,-2,0,2,4,6,8,10]) 
# plt.title("joke 100") 
# plt.show()

# plt.hist(joke_mean_ratings, bins =  [-10,-8,-6,-4,-2,0,2,4,6,8,10]) 
# plt.title("histogram") 
# plt.show()

In [ ]:
from sklearn import preprocessing
def pca(X):
    #Standardize the data (mean = 0 and variance = 1)
    X_std = preprocessing.scale(X, with_mean=False)

    # Calculate covariance matrix
    covariance_matrix = np.cov(np.transpose(X_std))

    # Calculate the eigenvectors and eigenvalues of the covariave matrix
    eig_val, eig_vec = np.linalg.eig(covariance_matrix)

    # Sort eig_value based on eig_val from highest to lowest
    abs_eig_val = np.abs(eig_val)
    indices = np.argsort(-abs_eig_val)
    abs_eig_val = abs_eig_val[indices]
    
    # Find top K eigen values that cover 85% variance
    K = abs_eig_val.shape[0]-1
    eig_sum = 0
    for i in range(abs_eig_val.shape[0]):
        eig_sum = eig_sum+abs_eig_val[i]
    #print("sum of eig value: ", eig_sum)

    eig_val_percent = abs_eig_val/eig_sum

    eig_percent_sum = 0
    for i in range(eig_val.shape[0]):
        eig_percent_sum = eig_percent_sum+eig_val_percent[i]
        if eig_percent_sum>=0.70:
            K = i
            print("K: ", K," %=", eig_percent_sum)
            break

    eig_vec = eig_vec[indices]

    eig_pairs = [(abs_eig_val[i], eig_vec[:,i]) for i in range(abs_eig_val.shape[0])]
    #print(eig_pairs)

    # Select the top K eig_vec
    selected_feature=np.array([ele[1] for ele in eig_pairs[:K]])

    # Get new data: Project original data to new dimension
    project_matrix = np.transpose(selected_feature)
    data=np.dot(X_std, project_matrix)
    # print("X_std.shape", X_std.shape, "project_matrix.shape", project_matrix.shape, "After PCA X shape: ", data.shape)
    return data

pca_X = pca(users_vec)
print(pca_X.shape)

plt.scatter(pca_X[:,0], pca_X[:,1], alpha=0.6)  # 绘制散点图，透明度为0.6（这样颜色浅一点，比较好看）
plt.show()

In [ ]:
from sklearn.neighbors import NearestNeighbors



neigh = NearestNeighbors(5)
neigh.fit(pca_X)


output_rating = np.full_like(test_userID, 0)
test_user_vec = pca_X[np.array(test_userID)-1]
test_user_vec_neighbors = neigh.kneighbors(test_user_vec, 5, return_distance=False)
print(test_user_vec_neighbors)


In [ ]:
# print(np.mean(users_vec[test_user_vec_neighbors[0]],axis=0)[0])
for i,user_neighbor in enumerate(test_user_vec_neighbors):
    m=np.mean(users_vec[user_neighbor], axis=0)[test_itemID[i]-1]
    output_rating[i]=m
print(output_rating)

In [ ]:
test_output=[['user_id-item_id','rating']]
for i in range(len(test_userID)):
    entry=str(test_userID[i])+'-'+str(test_itemID[i])
    value=output_rating[i]
    test_output.append([entry,value])
np.savetxt("output.csv", np.array(test_output, dtype=np.str), fmt='%s,%s', delimiter=",")